In [ ]:
import cv2
import numpy as np
from PIL import Image
from datasets.load import load_dataset
from pycocotools import mask as maskUtils

In [ ]:
dataset = load_dataset('coco_stuff', trust_remote_code=True)

In [ ]:
def transforms(examples):
    sem_rles = examples.pop('sem.rles')
    annotation = []
    for i in sem_rles:
        sem_rles = [
            {'size': size, 'counts': counts}
            for size, counts in zip(i['size'], i['counts'])
        ]
        annotation.append(maskUtils.decode(sem_rles))
    examples['annotation'] = annotation
    return examples

def visualize(example, colors):
    image = np.array(example['image'])
    categories = example['categories']
    masks = example['annotation']
    n = len(categories)
    for i in range(n):
        c = categories[i]
        color = colors[c]
        image[masks[..., i] == 1] = image[masks[..., i] == 1] // 2 + color // 2
    return image


dataset.set_transform(transforms)

names = dataset['train'].features['categories'].feature.names

colors = np.ones((92, 3), np.uint8) * 255
colors[:, 0] = np.linspace(0, 255, 92)
colors = cv2.cvtColor(colors[None], cv2.COLOR_HSV2RGB)[0]

example = dataset['train'][500]
Image.fromarray(visualize(example, colors))